In [1]:
import sys
import os
sys.path.append(os.path.abspath('../src'))
from foe_dataset import FOEDataset
import torch
from matplotlib import pyplot as plt
import numpy as np
%matplotlib widget

real_path = '../datasets/Finger/FOESamples'
synth_path = '../datasets/Finger/synfin/v1'

dataset = FOEDataset(real_path, patch_size=27, synth=False, subset="all")
# img = plt.imread(os.path.join(real_path,"Good","110.bmp"))
# fg = np.loadtxt(os.path.join(real_path,"Good","110.fg"),skiprows=1)
# gt = dataset.read_gt(os.path.join(real_path,"Good","110.gt"))
# X, Y, PATCH, LABEL, ANGLE = dataset.extract_patch_data(img, fg, gt)
# dataset.plot_grid(img, patch_data)
# plt.figure()
# plt.hist(ANGLE)
# for i in range(2000,2050):
#     plt.figure()
#     plt.imshow(PATCH[i]), print(LABEL[i], ANGLE[i])

# dataset = FOEDataset(synth_path, patch_size=27, synth=True)
# img = plt.imread(os.path.join(synth_path,"110.png"))
# fg = np.loadtxt(os.path.join(synth_path,"110.fg"),skiprows=1)
# gt = dataset.read_gt(os.path.join(synth_path,"110.gt"))
# dataset.plot_grid(img,dataset.extract_patch_data(img, fg, gt))

ModuleNotFoundError: No module named 'foe_dataset'

In [ ]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.cnn_layers = torch.nn.Sequential(
            # Defining a 2D convolution layer
            torch.nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(p=0.2),
            # Defining another 2D convolution layer
            torch.nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(p=0.2),
            # Defining another 2D convolution layer
            torch.nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(p=0.2),
            # Defining another 2D convolution layer
            torch.nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(4),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(p=0.2),
        )

        self.linear_layers = torch.nn.Sequential(
            torch.nn.Linear(4 * 3 * 3, 512),
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(512, 256),
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(256, 1)
        )
    
    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

    def init_weights(self,m):
        if type(m) == torch.nn.Linear:
            torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='leaky_relu')
            m.bias.data.fill_(0.01)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    train_loss, correct = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X = torch.stack(X)
        y = torch.tensor(y)
        y = y.view((-1, 1))
        idx = torch.randperm(X.size()[0])
        X = X[idx]
        y = y[idx]
        X, y = X.to(device, dtype=torch.float), y.to(device, dtype=torch.float)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y.float())
        train_loss += loss.item()
        #correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            #print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    train_loss /= len(dataloader)
    #correct /= len(dataloader.dataset)
    print(f"Training Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f} \n")
    return[train_loss, correct]

In [ ]:
def test(dataloader, model):
    model.eval()
    test_loss, correct = 0, 0
    predd, yy = [], []
    with torch.no_grad():
        for X, y in dataloader:
            X = torch.stack(X)
            y = torch.tensor(y)
            y = y.view((-1, 1))
            X, y = X.to(device, dtype=torch.float), y.to(device, dtype=torch.float)
            pred = model(X)
            predd.append(pred)
            yy.append(y)
            test_loss += loss_fn(pred, y.float()).item()
            #correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= len(dataloader)
    #correct /= len(dataloader.dataset)
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return[test_loss, correct, predd, yy]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
loss_fn = torch.nn.MSELoss()

N = len(dataset)
batch_size = 1
train_size = round(N*0.8)
val_size = round(N*0.2)
test_size = round(N*0.2)
training_data, validation_data = torch.utils.data.random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))

folds = 1
#test_dataloader =  torch.utils.data.DataLoader(test_data, batch_size=batch_size)
for f in range(folds):
    model = NeuralNetwork().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    #optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.8)
    #training_data, validation_data = torch.utils.data.random_split(training_validation_data, [train_size, val_size], generator=torch.Generator().manual_seed(42))
    train_dataloader =  torch.utils.data.DataLoader(training_data, batch_size=batch_size)
    val_dataloader =  torch.utils.data.DataLoader(validation_data, batch_size=batch_size)

    epochs = 50
    train_res = []
    test_res = []
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_ret = train(train_dataloader, model, loss_fn, optimizer)
        test_ret = test(val_dataloader, model)
        train_res.append(train_ret)
        test_res.append(test_ret)
    print("Done!")


In [ ]:
%matplotlib widget

from matplotlib import pyplot as plt
import numpy as np

train_res = np.array(train_res)
test_res = np.array(test_res)
plt.figure()
plt.plot(train_res[:,0])
plt.plot(test_res[:,0])
# plt.figure()
# plt.plot(train_res[:,1])
# plt.plot(test_res[:,1])

In [ ]:
test_loss, correct, predd, yy = test(val_dataloader, model)
aaa = np.array([])
bbb = np.array([])
for p in predd: aaa = np.append(aaa, p.cpu().data.numpy().flatten())
for y in yy: bbb = np.append(bbb, y.cpu().data.numpy().flatten())
ccc = np.power(np.mean(np.power((aaa-bbb),2)),0.5)

print(aaa.shape, bbb.shape, test_loss, ccc*180)
print(predd[0].transpose(0,1).max())
print(predd[0].transpose(0,1).min())
print(yy[0].transpose(0,1).max())
print(yy[0].transpose(0,1).min())

plt.figure()
hist_data = plt.hist(np.abs((aaa-bbb))*180,100)